In [1]:
from pdf2image import convert_from_path
import os
import pytesseract
from pytesseract import Output
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = 'C:\\Users\\Harle\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'

In [2]:
def plot(name, image):
    im_size = cv2.resize(image, (500, 800))
    cv2.imshow(name, im_size)
    if cv2.waitKey(0) & 0xff == 27: 
        cv2.destroyAllWindows()

In [ ]:
def convert_to_jpg():
    os.chdir("C:\\Users\\Harleen\\OneDrive\\Desktop\\Reciept Extraction\\dataset")
    data_folder="C:\\data_imgs\\"
    for file in os.listdir():
        file_name = file.strip(".pdf")
        images = convert_from_path(file, poppler_path=r"C:\Users\Harleen\AppData\Local\poppler-21.03.0\Library\bin")
        images[0].save(data_folder+file_name+".jpg", "JPEG", quality=80, optimize=True, progressive=True)
        print(file)       
    os.chdir("..")

In [ ]:
# def ocr():
#     os.chdir("C:\\Users\\Harleen\\OneDrive\\Desktop\\Reciept Extraction\\dataset\\data_imgs")
#     data_folder = "C:\\data_text\\"
#     for file in os.listdir():
#         with open(file, 'rb') as f:
#             image = cv2.imread(f)
#             img = get_grayscale(image)
#             #Tesseract
#             custom_config = r'--oem 3 --psm 6'
#             text = pytesseract.image_to_string(img,config=custom_config)
#             #Save Text
#             file_path = os.path.join(data_folder, os.path.splitext(file)[0]+".txt")
#             file = open(file_path, "w+")
#             file.write(text)
#             file.close()

In [4]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.fastNlMeansDenoising(image, None, 10, 10, 7, 15)

#simple thresholding
def simple_thresh(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#adaptive mean thresholding
def adaptmean_thresh(image):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 199, 5)

#adaptive gaussian thresholding
def adaptgauss_thresh(image):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 199, 5)

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#closing - dilation followed by an erosion
def closing(image):
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [ ]:
def ocr():
    os.chdir("C:\\Users\Harle\\OneDrive\Desktop\\Reciept Extraction\\dataset\\data_imgs")
    data_folder = "C:\\data_text\\"
    for file in os.listdir():
        # read image
        image = cv2.imread(file)
        # preprocessing (grayscale and ostu binarization)
        gray = get_grayscale(image)
        binary = simple_thresh(gray)
        preprocess_img = binary
        # tesseract
        custom_config = r'--oem 3 --psm 6'
        text = pytesseract.image_to_string(preprocess_img,config=custom_config)
        # save text to data folder
        file_path = os.path.join(data_folder, os.path.splitext(file)[0]+".txt")
        file = open(file_path, "w+")
        file.write(text)
        file.close()

In [10]:
image = cv2.imread(r"C:\Users\Harle\OneDrive - York St John University\YEAR 3\Work Based Project\Code\Text Localisation\test_img.jpg")

gray = get_grayscale(image)
# simple_thresh = simple_thresh(gray)
mean_thresh = adaptmean_thresh(gray)
gauss_thresh = adaptgauss_thresh(gray)
# opening = opening(gray)
# deskew = deskew(gray)
# noise = remove_noise(gray)

# plot("a", gauss_thresh)
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(gauss_thresh,config=custom_config)
boxes = pytesseract.image_to_boxes(gauss_thresh,config=custom_config)
results = pytesseract.image_to_data(gauss_thresh,output_type=Output.DICT)
table = pytesseract.image_to_data(gauss_thresh, config=custom_config)



In [14]:
# to generate text with boundign boxes as txt

for i in range(0, len(results["text"])):
	text = []
	# We can then extract the bounding box coordinates
	# of the text region from the current result
	x = results["left"][i]
	y = results["top"][i]
	w = results["width"][i]
	h = results["height"][i]
	text = results["text"][i]
	conf = int(float(results["conf"][i]))
	if conf > 70:
		print(text.upper(), ",", x,",", y,",", w,",", h)
		# text.append(text.upper(), ",", x,",", y,",", w,",", h)


11/30/2018 , 74 , 149 , 129 , 28
19:48 , 446 , 152 , 63 , 24
BEERHAUS , 231 , 216 , 104 , 25
CHECK: , 71 , 249 , 75 , 24
303004 , 177 , 249 , 80 , 25
TABLE: , 337 , 249 , 75 , 25
SERVER: , 71 , 281 , 89 , 25
ALLISON , 177 , 281 , 92 , 25
TERMINAL: , 71 , 313 , 115 , 26
30 , 204 , 314 , 25 , 24
BEERHAUS , 71 , 347 , 105 , 24
REGULAR , 203 , 387 , 92 , 53
1 , 153 , 446 , 6 , 23
CORONA , 177 , 445 , 78 , 24
7,00 , 457 , 446 , 50 , 24
2 , 150 , 478 , 12 , 24
18.00 , 445 , 479 , 62 , 24
9,00 , 203 , 510 , 52 , 24
1 , 152 , 544 , 6 , 24
DOGFISH , 176 , 542 , 92 , 29
90 , 283 , 543 , 25 , 24
IPA , 323 , 544 , 38 , 24
11.00 , 445 , 545 , 62 , 24
1 , 152 , 577 , 6 , 23
SAM , 176 , 577 , 39 , 23
SMITH , 229 , 576 , 65 , 24
CIDER , 309 , 576 , 65 , 25
9,00 , 456 , 577 , 51 , 24
2 , 149 , 609 , 12 , 23
MLK , 269 , 608 , 38 , 25
STOUT , 322 , 609 , 65 , 24
@ , 176 , 641 , 12 , 24
9.00 , 202 , 641 , 52 , 24
1 , 152 , 674 , 6 , 25
HITACHINO , 176 , 673 , 118 , 26
NEST , 310 , 674 , 49 , 25
76.00 , 41

In [11]:
print(text)

11/30/2018 19:48
BeerHaus
Check: 303004 Table: B+1
Server: Allison T,
Terminal: 30
BeerHaus
Regular Check =
1 Corona 7,00
2 Weihenstephaner 18.00
4 9,00
1 Dogfish 90 IPA 11.00
1 Sam Smith Cider 9,00
2 Outlaw Mlk Stout 18.00
@ 9.00
1 Hitachino Nest 13.00
Subtotal 76.00
Sales Tax 6.27
Total 82.27
Payments
Mastercard EMV 82.27
XXXXXXXXXXXXSOSZ - WALTER/JENS
MasterCard
CVM: Signature
Entry Mode: Chip
Auth Mode: Issuer
AID: A0000000041010
TVR: 0000008000 .
TAD: 011060900324000006£0000000000000
OOFF
TSI: E800
ARC: 00
Total Payments 82.27
Remaining Balance 0.00
Check Fully Authorized
seeseTip Not Includedersse
This tip guide is provided for
your convenience.
- weirdo sage tint mains
48%- 13.68
20%-15,20



In [ ]:
print(table)


# Metrics

https://towardsdatascience.com/evaluating-ocr-output-quality-with-character-error-rate-cer-and-word-error-rate-wer-853175297510

Here we evaluate two metrics. Character Error Rate (CER) and Word Error Rate (WER).

CER calculation is based on the concept of Levenshtein distance, where we count the minimum number of character-level operations required to transform the ground truth text (aka reference text) into the OCR output.

Since this project involves transcription of particular sequences (e.g., social security number, phone number, etc.), then the use of CER won't be relevant.
On the other hand, Word Error Rate might be more applicable if it involves the transcription of paragraphs and sentences of words with meaning (e.g., pages of books, newspapers).


In [3]:
import cv2
import pytesseract
import fastwer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [4]:
from platform import python_version

print(python_version())

3.9.9


In [34]:
ref = 'my name is kenneth'
output = 'myy nime iz kenneth'

# CER
print(fastwer.score_sent(output, ref, char_level=True))

# WER
print(fastwer.score_sent(output, ref))

16.6667
75.0


In [6]:
# Create empty dataframe to store output
df_output = pd.DataFrame(columns = ['img_filename', 'ocr_output'])

In [7]:
os.chdir("C:\\Users\\Harle\\OneDrive\\Desktop\\Reciept Extraction\\metrics\\sample_imgs")
for img in os.listdir():
    image = cv2.imread(img)
    gray = get_grayscale(image)
    custom_config = r'--oem 3 --psm 6'
    output = pytesseract.image_to_string(gray,config=custom_config)
    # print(output)
    dictionary = {'img_filename':img, 'ocr_output':str(output)}
    df_output = df_output.append(dictionary, ignore_index=True)

In [8]:
# Create new columns for reference, CER and WER
df_output['ref_text'] = ''
df_output['cer'] = ''
df_output['wer'] = ''

In [9]:
df_output

,img_filename,ocr_output,ref_text,cer,wer
0,bart-43253423.jpg,TVM No-.: Ho\nBART |\n\nsan Franeisco Int’l Ai...,,,
1,caffenero_20190831.jpg,Caffe Nero\n782 Edinburgh Airport Gate 12\nVAT...,,,
2,cheesecake-20191221_003.jpg,THE CHEESECAKE FACTORY\nLAS VEGAS\nOF27a TABLE...,,,
3,cowgirlcreamery-20190420_010.jpg,Cowgirl Creamery\n415-362-9354\nThe Ferry Plaz...,,,
4,gourmetmash_20190830.jpg,Makars Gourmet Mash Bar (West\nEnd)\n91 Shandw...,,,
5,haymarketedi_20190826.jpg,Haymarket\nEdinburgh\nEH12 SDS\n74179639~P0S-0...,,,
6,lebowski_20190828.jpg,"Lebouski’s\n""the dude abides’\nServer: MARCO 2...",,,
7,roxannes_599355.jpg,3\nRoxanne’s Cafe\n570 Powell Street\nSan fran...,,,
8,shakeshack_20181208_004.jpg,SHAKE SHACK\n3790 Las Vegas Blvd South\nHost: ...,,,
9,topgold-20191203.jpg,"TOPGOLF\n4627 Koval Lane\nLas Vegas, NV 89109\...",,,


In [26]:
# 1
df_output.loc[df_output['img_filename'] == 'bart-43253423.jpg', 'ref_text'] = '''TVM No-.: 5315
BART

san Francisco Int'l Airp
ort

497 North Link Road
DATE: O7/21/18

TIME: 10:11 PM

Debit Card Sale

CARD NO. 2825
AMOUNT $ 20.00
AUTHORIZATION DENIED
CONTACT BANK

Thanks for riding BART.'''

#2
df_output.loc[df_output['img_filename'] == 'caffenero_20190831.jpg', 'ref_text'] = '''Caffe Nero
782 Edinburgh Airport Gate 12
VAT: 795871659
1327 Barista 7
dK 30052
ake Away
1 Salted Caramel Cheesecake 2.95
1 Chai Latte 3.30
1 Latte Grande 3.05
Credit Card MPG GBP 9.30
1282516103619537900
1.06 VAT 20 % 6.35
Net Total: GBP 5.29
Subtotal GBP 9.30
Payment GBP 9.30
Change Due GBP 0.00
----------- Check Closed -----------
8/31/2019 11:15 AM
Tell us how we did today.
Visit www.mynerovisit com'''

#3
df_output.loc[df_output['img_filename'] == 'cheesecake-20191221_003.jpg', 'ref_text'] = '''THE CHEESECAKE FACTORY
LAS VEGAS
OF27a TABLE 82 #Party 1
BRANDON L SvrCk; 32 19:34 12/06/19
DINING ROOM
Separate checks: 2-of-4
| Great Basin Dr 7.50
1 Louisiana Chicken Pasta 18.50
1 Blue Moon Draft 7.50
1 Original Cheesecake 7.95
Sub Total: 41.45
Tax: 3.42
12/06 20:57 TOTAL : 44.87
Gratuity Not included
Suggested Gratuity:
22% 9.87
20% 8.97
18% 8.08
15% 6.73
We'd love to hear about your visit!
wow .ccfsurvey .com
Enter this code within 5 days:
1092-60171-02027
Join us for Brunch, Sat/Sun 10-2
For to-go orders, please visit
order thecheesecakefactory.com'''

#4
df_output.loc[df_output['img_filename'] == 'cowgirlcreamery-20190420_010.jpg', 'ref_text'] = '''Cowgirl Creamery
415-362-9354
The Ferry Plaza
One Embarcadero
TONY
Host: Gabino 04/08/2019
TONY 11:55 AM
40060
Far nouse GC 9.75
House Made Lemonade 2.95
Subtotal 12.70
Tax 1.08
Order Total 13.78
MC 13.78
Auth: NV97AY
Thank you for shopping at
Cowgirl Creamery!
Follow us on Instagram & Facebook
for the latest cheese news!
--- Check Closed ---'''

#5
df_output.loc[df_output['img_filename'] == 'gourmetmash_20190830.jpg', 'ref_text'] = '''Makars Gourmet Mash Bar (West
End)
91 Shandwick Place
Edinburgh, EH2 4SD
0131 228 5100
ORDER: Table 20
Cashier: Calli
30-Aug-2019 19:53:54
1 Lamb Shank £15.00
Bacon £0.00
1 Boar Sausage £12.00
Cheese £0.00
1 Stewarts Edinburgh Gold £4.00
2 Robert Burns Ale £9.00
1 Belhaven Craft Pilsner £4.00
2 Sticky Toffee £10.00
Total £54.00
VAT
% Net Tax Gross
20 45.00 9.00 54.00
EOTIS Ltd Reg# 342027 VAT No:
GB839829469
Online: https://eu.clover.com/r/
SOZHB8XAZKC3T
SOZHB8XAZKC3T
Order SOZHBBXAZKC3T
Clover Privacy Policy
https.//eu.clovet.com/pp/clover'''

#6
df_output.loc[df_output['img_filename'] == 'haymarketedi_20190826.jpg', 'ref_text'] = '''Haymarket
Edinburgh
EH12 SDS
ZA179639-P0S-04
Gary 26 Aug 2019 20: 16
Table: 26 c: 2 Acc No: 2747
4 Guinness Keg = 17.00
1 Steak Ale Pie = 11.00
| Chicken Mush Pie = 11.00
Product Group Summary
Food And Drink 39.00
20% VAT Net 32.50
20% VAT 6.50
20% VAT Total 39.00
Total £39.00
Nicholson's Gift Card - tue ° orfect
gift for pub lovers! wist °: a memuer
of our tea today ci gy ta:
wuw.nicholscnssuos.co.t+ iftcards
Tel 0121 228 2!
VAT 2¢2 185'''

#7
df_output.loc[df_output['img_filename'] == 'lebowski_20190828.jpg', 'ref_text'] = '''Lebowski’s
"the dude abides’
Server: MARCO 28/08/2019
Cashier: MICHEAL
Table 14/1 9:49 PM
Guests: 2 30044
MAC WAIN 9.95

FISH & CHIPS 10.95
Bell field ipa (4 @4.60) 18.40
Guinness Pt (2 @4.80) 9.60
Subtotal 40.75
Tax 8.15
Total 48.90
Balance Due 48.90

18 Morrison Street

Edinburgh, EH3 8BJ

WiFi Password: lebowskis1013

Tel No: 0131 466 1779

VAT No: 170 5101 51

www. lebowskis.co.uk'''

#8
df_output.loc[df_output['img_filename'] == 'roxannes_599355.jpg', 'ref_text'] = '''Roxanne’s Cafe
570 Powell Street
San francisco, California
Tel: (415) 989-5555
Check #: 599355
Server: Masha Date: 05/12/2019
Table: 21 Time: 09:29
Client: 1
1 Coffee 2.75
1 Grande 15.00
SUB-TOTAL : 17.75
S.F.H.O (0): 0.53
Sales Tax1: 1.60
Total: 19.88
For General Comments About
Your Dining Experience
Please Contact Us
We Are Obligated to
Collect a Surcharge for
the SF Health Care
Security Ordinance'''

#9
df_output.loc[df_output['img_filename'] == 'shakeshack_20181208_004.jpg', 'ref_text'] = '''SHAKE SHACK
3790 Las Vegas Blvd South
Host: Lisa 12/02/2018
166 WALTER 11:44 AM
60042
DBL ShackBurger 8.69
Bacon Cheese Fries 4.89
Shake 5.49
Vanilla Shake
Subtotal 19.07
Tax 1.57
To Stay Total 20.64
MasterCard #XXXXXXXXXXXX2825 20.64
Auth: NGJJ58
We wanna hear ya! Take our survey
for $5 off your next $20 App order.
http://bit.1ly/shack-survey-1130
--- Check Closed ---'''

#10
df_output.loc[df_output['img_filename'] == 'topgold-20191203.jpg', 'ref_text'] = '''TOPGOLF
4627 Koval Lane
Las Vegas, NV 89109

Check 465 Tab 111/WALTER
Caitlyn B. 12/3/2019
Guests 1 9:13 PM
4 Membership Charge (5.00) 20.00

Topgolf Gameplay 63.00

Bud Light Draft 7.25

Lagunitas IPA Pint 8.25

SA New England IPA 8.25

Heineken Can 8.25
Subtotal 115.00
Tax 2.64
TOTAL 117.64
BALANCE DUE 117.64

Thank you for visiting Topgolf! For

Upcoming events and promotions visit

Topgolf.com and follow us @topgolf

on Facebook, Twitter or Instagram.'''

In [28]:
# Replace new lines in output/ref text with space
df_output['ocr_output'] = df_output['ocr_output'].apply(lambda x: x.replace('\n',' ').upper())
df_output['ref_text'] = df_output['ref_text'].apply(lambda x: x.replace('\n',' ').upper())


In [32]:
for index, row in df_output.iterrows():
  filename = row['img_filename']
  ref = row['ref_text']
  output = row['ocr_output']
  cer = fastwer.score_sent(output, ref, char_level=True)
  wer = fastwer.score_sent(output, ref, char_level=False)
  df_output.loc[df_output['img_filename'] == filename, 'cer'] = round(cer,2) # Round value to 2 decimal places
  df_output.loc[df_output['img_filename'] == filename, 'wer'] = round(wer,2)

df_output

,img_filename,ocr_output,ref_text,cer,wer
0,bart-43253423.jpg,TVM NO-.: HO BART | SAN FRANEISCO INT’L AIRP ...,TVM NO-.: 5315 BART SAN FRANCISCO INT'L AIRP ...,8.21,19.51
1,caffenero_20190831.jpg,CAFFE NERO 782 EDINBURGH AIRPORT GATE 12 VAT: ...,CAFFE NERO 782 EDINBURGH AIRPORT GATE 12 VAT: ...,1.72,5.71
2,cheesecake-20191221_003.jpg,THE CHEESECAKE FACTORY LAS VEGAS OF27A TABLE 6...,THE CHEESECAKE FACTORY LAS VEGAS OF27A TABLE 8...,1.43,6.45
3,cowgirlcreamery-20190420_010.jpg,COWGIRL CREAMERY 415-362-9354 THE FERRY PLAZA ...,COWGIRL CREAMERY 415-362-9354 THE FERRY PLAZA ...,4.07,10.53
4,gourmetmash_20190830.jpg,MAKARS GOURMET MASH BAR (WEST END) 91 SHANDWIC...,MAKARS GOURMET MASH BAR (WEST END) 91 SHANDWIC...,0.54,1.23
5,haymarketedi_20190826.jpg,HAYMARKET EDINBURGH EH12 SDS 74179639~P0S-04 G...,HAYMARKET EDINBURGH EH12 SDS ZA179639-P0S-04 G...,1.35,3.49
6,lebowski_20190828.jpg,"LEBOUSKI’S ""THE DUDE ABIDES’ SERVER: MARCO 28/...","LEBOWSKI’S ""THE DUDE ABIDES’ SERVER: MARCO 28/...",1.28,5.63
7,roxannes_599355.jpg,3 ROXANNE’S CAFE 570 POWELL STREET SAN FRANCIS...,ROXANNE’S CAFE 570 POWELL STREET SAN FRANCISCO...,4.46,7.69
8,shakeshack_20181208_004.jpg,SHAKE SHACK 3790 LAS VEGAS BLVD SOUTH HOST: LI...,SHAKE SHACK 3790 LAS VEGAS BLVD SOUTH HOST: LI...,0.27,0.0
9,topgold-20191203.jpg,"TOPGOLF 4627 KOVAL LANE LAS VEGAS, NV 89109 C...","TOPGOLF 4627 KOVAL LANE LAS VEGAS, NV 89109 C...",0.65,1.2


In [38]:
# Overall performances
mean_cer = df_output['cer'].mean()
mean_wer = df_output['wer'].mean()
print(f'Mean CER = {round(mean_cer,3)}%, Mean WER = {round(mean_wer,3)}%')

Mean CER = 2.398%, Mean WER = 6.144%


#